In [204]:
import matplotlib.pyplot as plt
import pandas as pd
from timple.timedelta import strftimedelta
import fastf1 as ff1
import fastf1.plotting
from fastf1.core import Laps
import plotly.express as px
import numpy as np
import warnings
import re
import math
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_rows = 75
pd.options.display.max_columns = 99

fastf1.Cache.enable_cache('../cache')
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None, misc_mpl_mods=False)

In [205]:
# maListe = ['FP1', 'FP2', 'FP3']
# dffp = pd.DataFrame(columns=['Time', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'PitOutTime',
#        'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
#        'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
#        'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
#        'Compound', 'TyreLife', 'FreshTyre', 'LapStartTime', 'Team', 'Driver',
#        'TrackStatus', 'IsAccurate', 'LapStartDate'])

# for i in maListe:
#     fp = ff1.get_session(2022, 7, i)
#     fp.load(weather=True, telemetry=True)
#     df1 = fp.laps.pick_quicklaps()
#     df1 = pd.DataFrame(df1)
#     df1['Session'] = i
#     dffp = pd.concat([dffp, df1])

dffp = pd.read_csv('https://f1-jedha-bucket.s3.eu-west-3.amazonaws.com/data/full_data-round_1.csv', index_col=0)
dffp

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,Session
2,0 days 00:21:57.674000,1,0 days 00:01:37.766000,3.0,1.0,NaN,NaN,0 days 00:00:30.695000,0 days 00:00:41.580000,0 days 00:00:25.491000,0 days 00:20:50.603000,0 days 00:21:32.183000,0 days 00:21:57.674000,228.0,263.0,176.0,306.0,False,HARD,3.0,True,0 days 00:20:19.908000,Red Bull Racing,VER,2,True,2022-03-18 12:05:19.921000,Practice 1
5,0 days 00:39:39.617000,1,0 days 00:01:34.783000,6.0,2.0,NaN,NaN,0 days 00:00:30.119000,0 days 00:00:40.789000,0 days 00:00:23.875000,0 days 00:38:34.953000,0 days 00:39:15.742000,0 days 00:39:39.617000,232.0,263.0,278.0,309.0,False,MEDIUM,3.0,True,0 days 00:38:04.834000,Red Bull Racing,VER,1,True,2022-03-18 12:23:04.847000,Practice 1
7,0 days 00:43:23.119000,1,0 days 00:01:34.742000,8.0,2.0,NaN,NaN,0 days 00:00:30.224000,0 days 00:00:40.815000,0 days 00:00:23.703000,0 days 00:42:18.601000,0 days 00:42:59.416000,0 days 00:43:23.119000,231.0,263.0,279.0,310.0,True,MEDIUM,5.0,True,0 days 00:41:48.377000,Red Bull Racing,VER,1,True,2022-03-18 12:26:48.390000,Practice 1
11,0 days 01:00:16.005000,1,0 days 00:01:38.702000,12.0,4.0,NaN,NaN,0 days 00:00:31.520000,0 days 00:00:42.499000,0 days 00:00:24.683000,0 days 00:59:08.823000,0 days 00:59:51.322000,0 days 01:00:16.005000,NaN,255.0,264.0,291.0,False,MEDIUM,9.0,False,0 days 00:58:37.303000,Red Bull Racing,VER,1,True,2022-03-18 12:43:37.316000,Practice 1
12,0 days 01:01:55.195000,1,0 days 00:01:39.190000,13.0,4.0,NaN,NaN,0 days 00:00:31.789000,0 days 00:00:42.890000,0 days 00:00:24.511000,0 days 01:00:47.794000,0 days 01:01:30.684000,0 days 01:01:55.195000,224.0,255.0,265.0,285.0,False,MEDIUM,10.0,False,0 days 01:00:16.005000,Red Bull Racing,VER,1,True,2022-03-18 12:45:16.018000,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,0 days 00:35:26.406000,77,0 days 00:01:35.365000,7.0,2.0,NaN,NaN,0 days 00:00:30.481000,0 days 00:00:41.119000,0 days 00:00:23.765000,0 days 00:34:21.522000,0 days 00:35:02.641000,0 days 00:35:26.406000,235.0,265.0,281.0,305.0,False,SOFT,5.0,True,0 days 00:33:51.041000,Alfa Romeo,BOT,1,True,2022-03-19 12:19:10.662000,Practice 3
315,0 days 00:39:19.139000,77,0 days 00:01:34.806000,9.0,2.0,NaN,NaN,0 days 00:00:30.338000,0 days 00:00:40.695000,0 days 00:00:23.773000,0 days 00:38:14.671000,0 days 00:38:55.366000,0 days 00:39:19.139000,236.0,266.0,282.0,309.0,False,SOFT,7.0,True,0 days 00:37:44.333000,Alfa Romeo,BOT,1,True,2022-03-19 12:23:03.954000,Practice 3
319,0 days 00:58:18.855000,77,0 days 00:01:33.733000,13.0,4.0,NaN,NaN,0 days 00:00:30.097000,0 days 00:00:40.138000,0 days 00:00:23.498000,0 days 00:57:15.219000,0 days 00:57:55.357000,0 days 00:58:18.855000,234.0,266.0,282.0,310.0,True,SOFT,3.0,True,0 days 00:56:45.122000,Alfa Romeo,BOT,1,True,2022-03-19 12:42:04.743000,Practice 3
321,0 days 01:02:03.424000,77,0 days 00:01:33.889000,15.0,4.0,NaN,NaN,0 days 00:00:29.995000,0 days 00:00:40.405000,0 days 00:00:23.489000,0 days 01:00:59.530000,0 days 01:01:39.935000,0 days 01:02:03.424000,236.0,267.0,282.0,310.0,False,SOFT,5.0,True,0 days 01:00:29.535000,Alfa Romeo,BOT,1,True,2022-03-19 12:45:49.156000,Practice 3


In [206]:
dffp = dffp.drop(dffp.columns[[0,5,6,7,8,9,10,11,12,13,14,15,16,17,21,25,26]], axis=1)
dffp

,DriverNumber,LapTime,LapNumber,Stint,Compound,TyreLife,FreshTyre,Team,Driver,TrackStatus,Session
2,1,0 days 00:01:37.766000,3.0,1.0,HARD,3.0,True,Red Bull Racing,VER,2,Practice 1
5,1,0 days 00:01:34.783000,6.0,2.0,MEDIUM,3.0,True,Red Bull Racing,VER,1,Practice 1
7,1,0 days 00:01:34.742000,8.0,2.0,MEDIUM,5.0,True,Red Bull Racing,VER,1,Practice 1
11,1,0 days 00:01:38.702000,12.0,4.0,MEDIUM,9.0,False,Red Bull Racing,VER,1,Practice 1
12,1,0 days 00:01:39.190000,13.0,4.0,MEDIUM,10.0,False,Red Bull Racing,VER,1,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...
313,77,0 days 00:01:35.365000,7.0,2.0,SOFT,5.0,True,Alfa Romeo,BOT,1,Practice 3
315,77,0 days 00:01:34.806000,9.0,2.0,SOFT,7.0,True,Alfa Romeo,BOT,1,Practice 3
319,77,0 days 00:01:33.733000,13.0,4.0,SOFT,3.0,True,Alfa Romeo,BOT,1,Practice 3
321,77,0 days 00:01:33.889000,15.0,4.0,SOFT,5.0,True,Alfa Romeo,BOT,1,Practice 3


In [207]:
dffp_temp = dffp.copy()
dffp_temp['LapTime'] = dffp_temp['LapTime'].apply(pd.to_timedelta)
dffp_times = dffp_temp.groupby('Compound')['LapTime'].mean()
dffp_times = pd.DataFrame(dffp_times).reset_index()
dffp_times

,Compound,LapTime
0,HARD,0 days 00:01:37.916250
1,MEDIUM,0 days 00:01:36.943102941
2,SOFT,0 days 00:01:35.426606741


In [208]:
hard_to_soft = (dffp_times['LapTime'].iloc[0] - dffp_times['LapTime'].iloc[2]) * 0.75
hard_to_soft

Timedelta('0 days 00:00:01.867232444')

In [209]:
medium_to_soft = (dffp_times['LapTime'].iloc[1] - dffp_times['LapTime'].iloc[2]) * 0.75
medium_to_soft

Timedelta('0 days 00:00:01.137372150')

In [210]:
soft_baseline = 0.0

In [211]:
dffp = dffp[dffp['TyreLife'].notna()]
dffp

,DriverNumber,LapTime,LapNumber,Stint,Compound,TyreLife,FreshTyre,Team,Driver,TrackStatus,Session
2,1,0 days 00:01:37.766000,3.0,1.0,HARD,3.0,True,Red Bull Racing,VER,2,Practice 1
5,1,0 days 00:01:34.783000,6.0,2.0,MEDIUM,3.0,True,Red Bull Racing,VER,1,Practice 1
7,1,0 days 00:01:34.742000,8.0,2.0,MEDIUM,5.0,True,Red Bull Racing,VER,1,Practice 1
11,1,0 days 00:01:38.702000,12.0,4.0,MEDIUM,9.0,False,Red Bull Racing,VER,1,Practice 1
12,1,0 days 00:01:39.190000,13.0,4.0,MEDIUM,10.0,False,Red Bull Racing,VER,1,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...
313,77,0 days 00:01:35.365000,7.0,2.0,SOFT,5.0,True,Alfa Romeo,BOT,1,Practice 3
315,77,0 days 00:01:34.806000,9.0,2.0,SOFT,7.0,True,Alfa Romeo,BOT,1,Practice 3
319,77,0 days 00:01:33.733000,13.0,4.0,SOFT,3.0,True,Alfa Romeo,BOT,1,Practice 3
321,77,0 days 00:01:33.889000,15.0,4.0,SOFT,5.0,True,Alfa Romeo,BOT,1,Practice 3


In [212]:
dffp['LapTime'] = dffp['LapTime'].apply(pd.to_timedelta)

/tmp/ipykernel_1036451/467148075.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [213]:
dffp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 2 to 323
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   DriverNumber  357 non-null    int64          
 1   LapTime       357 non-null    timedelta64[ns]
 2   LapNumber     357 non-null    float64        
 3   Stint         357 non-null    float64        
 4   Compound      357 non-null    object         
 5   TyreLife      357 non-null    float64        
 6   FreshTyre     347 non-null    object         
 7   Team          357 non-null    object         
 8   Driver        357 non-null    object         
 9   TrackStatus   357 non-null    int64          
 10  Session       357 non-null    object         
dtypes: float64(3), int64(2), object(5), timedelta64[ns](1)
memory usage: 33.5+ KB


In [214]:
#Convertir temps en seconde (script de Bérenger : OK)

def time_to_seconds(string):
    array = re.findall(r'[0-9]+', str(string))
    array = array[1:]
    array[0] = int(array[0]) * 3600
    array[1] = int(array[1]) * 60
    try : 
        array = float(str(array[0] + array[1] + int(array[2])) + "." + array[3])
        return array
    except :
        return float(str(int(array[-2]) + int(array[-1])) + ".0")

dffp["LapTimeSeconds"] = dffp["LapTime"].apply(time_to_seconds)

/tmp/ipykernel_1036451/2241231510.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [215]:
compound_colors = {
    'SOFT': '#FF3333',
    'MEDIUM': '#FFF200',
    'HARD': '#EBEBEB',
}

In [216]:
px.scatter(dffp, 
            x=dffp['TyreLife'] , 
            y='LapTimeSeconds', 
            color='Compound',
            color_discrete_map=compound_colors, 
            width=1000, height=700,
            template='plotly_dark',
            trendline="lowess",
            )

In [217]:
dffp_full = dffp.groupby(['TyreLife','Compound'] ,as_index=False)[['LapTimeSeconds']].mean()
dffp_full = dffp_full.sort_values(by= ['Compound','TyreLife'])
dffp_full

,TyreLife,Compound,LapTimeSeconds
2,2.0,HARD,97.555833
5,3.0,HARD,97.985000
8,4.0,HARD,97.790714
11,5.0,HARD,98.070250
14,6.0,HARD,98.193300
17,7.0,HARD,98.093571
20,8.0,HARD,97.696500
23,9.0,HARD,98.019200
26,10.0,HARD,97.565000
29,11.0,HARD,97.176500


In [218]:
dffp_filtered = dffp_full.sort_values(by= ['Compound','TyreLife'])
dffp_filtered = dffp_filtered[dffp_filtered['TyreLife'] > 5.0]
dffp_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 14 to 40
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   TyreLife        30 non-null     float64
 1   Compound        30 non-null     object 
 2   LapTimeSeconds  30 non-null     float64
dtypes: float64(2), object(1)
memory usage: 960.0+ bytes


In [219]:
dffp_soft = dffp_filtered[dffp_filtered['Compound'] == 'SOFT'].reset_index(drop=True)
dffp_medium = dffp_filtered[dffp_filtered['Compound'] == 'MEDIUM'].reset_index(drop=True)
dffp_hard = dffp_filtered[dffp_filtered['Compound'] == 'HARD'].reset_index(drop=True)

In [303]:
diff_soft = (dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0]) * 0.75
diff_soft

1.969673076923069

In [304]:
diff_medium = (dffp_medium['LapTimeSeconds'].iloc[-1] - dffp_medium['LapTimeSeconds'].iloc[0]) * 0.75
diff_medium

2.33092499999999

In [222]:
diff_hard = dffp_hard['LapTimeSeconds'].iloc[-1] - dffp_hard['LapTimeSeconds'].iloc[0]
diff_hard

0.07170000000000698

In [223]:
dict_nb_laps = {1: 57, 2: 50, 3: 58, 4: 63, 5: 57, 6: 66, 7: 78, 8: 51, 9: 70, 10: 52, 11: 71, 12: 53, 13: 70, 14: 44, 15: 72, 16: 53, 17: 61, 18: 53, 19: 56, 20: 71, 21: 71, 22: 58}
fuel_per_lap_kg = 105 / dict_nb_laps[1]
mean_laps = 60
time_gain_per_kg = 0.035
laps_coeff = mean_laps / dict_nb_laps[1]
time_gained_per_lap = laps_coeff * time_gain_per_kg * fuel_per_lap_kg
time_gained_per_lap

0.06786703601108034

In [305]:
dffp_soft['FuelGain'] = time_gained_per_lap
dffp_soft['FuelGain'] = dffp_soft['FuelGain'].cumsum()
dffp_soft['AdjustedTime'] = dffp_soft['LapTimeSeconds'] + dffp_soft['FuelGain']

diff_soft = (dffp_soft['AdjustedTime'].iloc[-1] - dffp_soft['AdjustedTime'].iloc[0]) * 0.75
diff_soft_per_lap = diff_soft / len(dffp_soft)
diff_soft_per_lap

0.24277755699978593

In [306]:
dffp_soft

,TyreLife,Compound,LapTimeSeconds,FuelGain,AdjustedTime
0,6.0,SOFT,94.997769,0.067867,95.065636
1,7.0,SOFT,95.686500,0.135734,95.822234
2,8.0,SOFT,96.631600,0.203601,96.835201
3,9.0,SOFT,96.911667,0.271468,97.183135
4,10.0,SOFT,97.650385,0.339335,97.989720
5,11.0,SOFT,97.199833,0.407202,97.607036
6,12.0,SOFT,97.081167,0.475069,97.556236
7,13.0,SOFT,98.170667,0.542936,98.713603
8,14.0,SOFT,97.916000,0.610803,98.526803
9,15.0,SOFT,97.624000,0.678670,98.302670


In [307]:
laptime_min_series = dffp_full[dffp_full['Compound'] == 'SOFT'].sort_values('LapTimeSeconds', ascending=True).iloc[0]
laptime_min = {'TyreLife': laptime_min_series['TyreLife'], 'LapTimeSeconds': laptime_min_series['LapTimeSeconds']}
laptime_min

{'TyreLife': 3.0, 'LapTimeSeconds': 93.96565909090909}

In [308]:
laptime_min_calc_soft =  (laptime_min['LapTimeSeconds'] - ((laptime_min['TyreLife'] - 1) * diff_soft_per_lap)) + (dict_nb_laps[1] * time_gained_per_lap)
laptime_min_calc_soft

97.3485250295411

In [309]:
laptime_min_calc_medium = (laptime_min_calc_soft + time_to_seconds(medium_to_soft))
laptime_min_calc_medium

98.4858971795411

In [310]:
laptime_min_calc_hard = laptime_min_calc_soft + time_to_seconds(hard_to_soft)
laptime_min_calc_hard

99.21575747354109

In [311]:
dc = 0.1
dr = 0.11329619326656674


In [312]:

dr = dr * (1 + dc)
dr

0.12462581259322343

In [324]:
list_times_soft = []
dc = 0.1
dr = diff_soft_per_lap

for i in range(1, dict_nb_laps[1] + 1):
    dr = dr * (1 + dc)
    list_times_soft.append(dr)
    
list_times_soft[:5]

[0.2670553126997645,
 0.293760843969741,
 0.32313692836671515,
 0.3554506212033867,
 0.3909956833237254]

In [325]:
df_times = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])



df_times['Lap'] = list(range(1, dict_nb_laps[1] + 1))
df_times['Tyre'] = 'SOFT'
df_times['LapTimeSeconds'] = laptime_min_calc_soft
df_times['DeltaDeg'] = list_times_soft
df_times['FuelDeg'] = time_gained_per_lap * df_times['Lap']
df_times['AdjustedTime'] = df_times.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times['FinalLapTime'] = df_times.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,SOFT,97.348525,0.267055,0.067867,97.615580,97.547713
1,2,SOFT,97.348525,0.293761,0.135734,97.642286,97.506552
2,3,SOFT,97.348525,0.323137,0.203601,97.671662,97.468061
3,4,SOFT,97.348525,0.355451,0.271468,97.703976,97.432508
4,5,SOFT,97.348525,0.390996,0.339335,97.739521,97.400186
5,6,SOFT,97.348525,0.430095,0.407202,97.778620,97.371418
6,7,SOFT,97.348525,0.473105,0.475069,97.821630,97.346561
7,8,SOFT,97.348525,0.520415,0.542936,97.868940,97.326004
8,9,SOFT,97.348525,0.572457,0.610803,97.920982,97.310178
9,10,SOFT,97.348525,0.629702,0.678670,97.978227,97.299557


In [326]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_times['Lap'].values, y=df_times['AdjustedTime'].values, mode='lines'))

fig.update_layout(width=1000, height=700, template='plotly_dark')
fig.show()

In [235]:
#ajout tours manquant en soft

lapsS = pd.DataFrame({'TyreLife': [16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                    41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78], 'Compound' : 'SOFT'})

lst = [dffp.iloc[40,2]*1.007]
res = dffp.iloc[40,2]*1.007
for i, row in lapsS.iloc[0:].iterrows():
    res = (res * 1.007)
    lst.append(res)

lapsS['LapTime'] = pd.Series(lst)

In [236]:
#ajout tours manquant en medium

lapsM = pd.DataFrame({'TyreLife': [16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                    41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78], 'Compound' : 'MEDIUM'})

lst = [dffp.iloc[39,2]*1.005]
res = dffp.iloc[39,2]*1.005
for i, row in lapsM.iloc[0:].iterrows():
    res = (res * 1.005)
    lst.append(res)

lapsM['LapTime'] = pd.Series(lst)

In [237]:
#ajout tours manquant en hard

lapsH = pd.DataFrame({'TyreLife': [16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                    41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78], 'Compound' : 'HARD'})

lst = [dffp.iloc[39,2]*1.003]
res = dffp.iloc[39,2]*1.003
for i, row in lapsH.iloc[0:].iterrows():
    res = (res * 1.003)
    lst.append(res)

lapsH['LapTime'] = pd.Series(lst)

In [238]:
#ajout tours 1 (et 2 pour les hards) à voir si il n'en manque pas pour d'autres GP

lap1S = pd.DataFrame({'TyreLife': [1], 'Compound' : 'SOFT', 'LapTime': [dffp.iloc[1,2]+1]})
lap1M = pd.DataFrame({'TyreLife': [1], 'Compound' : 'MEDIUM', 'LapTime': [dffp.iloc[0,2]+1]})
lap1H = pd.DataFrame({'TyreLife': [1], 'Compound' : 'HARD', 'LapTime': [dffp.iloc[2,2]+2]})
lap2H = pd.DataFrame({'TyreLife': [2], 'Compound' : 'HARD', 'LapTime': [dffp.iloc[2,2]+0.5]})

In [239]:
alllaps = pd.concat([dffp, lapsH, lapsM, lapsS,lap1H,lap1M,lap1S,lap2H ])

In [240]:
alllaps = alllaps.sort_values(by= ['Compound','TyreLife'])

In [241]:
alllaps

,DriverNumber,LapTime,LapNumber,Stint,Compound,TyreLife,FreshTyre,Team,Driver,TrackStatus,Session,LapTimeSeconds
0,NaN,10.0,NaN,NaN,HARD,1.0,NaN,NaN,NaN,NaN,NaN,NaN
24,55.0,0 days 00:01:38.745000,2.0,1.0,HARD,2.0,True,Ferrari,SAI,2.0,Practice 1,98.745
163,22.0,0 days 00:01:39.255000,2.0,1.0,HARD,2.0,True,AlphaTauri,TSU,1.0,Practice 1,99.255
385,4.0,0 days 00:01:35.486000,2.0,1.0,HARD,2.0,True,McLaren,NOR,1.0,Practice 2,95.486
71,31.0,0 days 00:01:36.634000,2.0,1.0,HARD,2.0,True,Alpine,OCO,1.0,Practice 3,96.634
...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,9.055032,NaN,NaN,SOFT,74.0,NaN,NaN,NaN,NaN,NaN,NaN
59,NaN,9.118418,NaN,NaN,SOFT,75.0,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,9.182247,NaN,NaN,SOFT,76.0,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,9.246522,NaN,NaN,SOFT,77.0,NaN,NaN,NaN,NaN,NaN,NaN
